# RAG の仕組みを自作して理解するハンズオン

新宿区が公開している「[新宿区くらしのガイド](https://www.city.shinjuku.lg.jp/kusei/snjkguide202404.html)」（20個程度の PDF 文書）に基づいて、区民の質問に回答するコンシェルジュ AI を作成します。

ここでは、質問の回答に必要な PDF をファイル単位で検索・特定した後に、トップ3の PDF をそのままプロンプトに埋め込んで回答を生成するアプローチを取ります。

質問に関連した PDF は次の方法で検索します。

- 各 PDF のサマリーテキストを埋め込みベクトル（サマリーベクトル）に変換しておく
- 質問文から「検索用クエリー」を生成して、これを埋め込みベクトル（クエリーベクトル）に変換する
- クエリーベクトルとサマリーベクトル類似度から関連性が高い PDF を特定する

また、質問者のプロファイルや過去の質問を補助的に入力することで、パーソナライズされた回答を生成する仕組みも実装します。

## 事前準備

### 環境の初期化

In [1]:
import os
import pickle
import requests

import numpy as np
import vertexai
from google import genai
from google.cloud import storage
from google.genai import types
from IPython.display import Markdown, display

# プロジェクト設定
[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

# SDK と Google Gen AI SDK クライアントの初期化
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

# PDF を保存するバケットを作成
BACKET = f'{PROJECT_ID}_ragdata'
!gsutil ls -b gs://{BACKET} || gsutil mb -l {LOCATION} gs://{BACKET}

gs://etsuji-15pro-poc_ragdata/


### 新宿区の公共ガイド PDF を GCS バケットに保存

In [2]:
pdf_dict = {
    '区長の挨拶': '000391171.pdf',
    '引っ越しするとき': '000391172.pdf',
    'もくじ': '000391174.pdf',
    'いざというときに備えて': '000391175.pdf',
    'エリアシティウォーク': '000391176.pdf',
    '公的施設等一覧': '000391178.pdf',
    '主な区立施設の設備一覧': '000391179.pdf',
    '相談': '000391180.pdf',
    '区役所と特別出張所': '000394761.pdf',
    '緊急のとき／安心安全': '000391182.pdf',
    '手続き／届出': '000391183.pdf',
    '健康／衛生': '000391184.pdf',
    '高齢者福祉': '000391185.pdf',
    '介護保険': '000391187.pdf',
    '障害者福祉': '000391189.pdf',
    'その他の福祉': '000391192.pdf',
    '子ども／教育': '000391194.pdf',
    '文化／スポーツ': '000391197.pdf',
    'ごみ／環境／リサイクル': '000391199.pdf',
    '生活環境': '000391201.pdf',
    '住宅／まちづくり': '000391203.pdf',
    '仕事／経営': '000391205.pdf',
    '協働／身近な行政': '000391206.pdf',
    'わたしの便利帳2024年版　さくいん（索引）': '000391177.pdf',
    '医療機関リスト': '000391207.pdf'
}

# Storage クライアントとバケットの準備
storage_client = storage.Client()
bucket = storage_client.bucket(BACKET)

print(f'Starting download and upload to gs://{BACKET}/pdf/ ...')

for title, filename in pdf_dict.items():
    url = f'https://www.city.shinjuku.lg.jp/content/{filename}'
    
    try:
        # PDFをダウンロード
        response = requests.get(url)
        response.raise_for_status() # エラーがあれば例外を発生させる
        
        # 保存先のパスを指定 (pdf/フォルダ配下)
        blob = bucket.blob(f'pdf/{filename}')
        
        # メモリ上のコンテンツを直接アップロード
        blob.upload_from_string(response.content, content_type='application/pdf')
        
        print(f'Succeeded: {title} ({filename})')
        
    except Exception as e:
        print(f'Failed: {title} ({filename}) - Reason: {e}')

print('\nAll processes are complete.')

Starting download and upload to gs://etsuji-15pro-poc_ragdata/pdf/ ...
Succeeded: 区長の挨拶 (000391171.pdf)
Succeeded: 引っ越しするとき (000391172.pdf)
Succeeded: もくじ (000391174.pdf)
Succeeded: いざというときに備えて (000391175.pdf)
Succeeded: エリアシティウォーク (000391176.pdf)
Succeeded: 公的施設等一覧 (000391178.pdf)
Succeeded: 主な区立施設の設備一覧 (000391179.pdf)
Succeeded: 相談 (000391180.pdf)
Succeeded: 区役所と特別出張所 (000394761.pdf)
Succeeded: 緊急のとき／安心安全 (000391182.pdf)
Succeeded: 手続き／届出 (000391183.pdf)
Succeeded: 健康／衛生 (000391184.pdf)
Succeeded: 高齢者福祉 (000391185.pdf)
Succeeded: 介護保険 (000391187.pdf)
Succeeded: 障害者福祉 (000391189.pdf)
Succeeded: その他の福祉 (000391192.pdf)
Succeeded: 子ども／教育 (000391194.pdf)
Succeeded: 文化／スポーツ (000391197.pdf)
Succeeded: ごみ／環境／リサイクル (000391199.pdf)
Succeeded: 生活環境 (000391201.pdf)
Succeeded: 住宅／まちづくり (000391203.pdf)
Succeeded: 仕事／経営 (000391205.pdf)
Succeeded: 協働／身近な行政 (000391206.pdf)
Succeeded: わたしの便利帳2024年版　さくいん（索引） (000391177.pdf)
Succeeded: 医療機関リスト (000391207.pdf)

All processes are complete.


## PDF 検索用の埋め込みベクトルを生成

### 各 PDF のサマリーテキストを生成

In [3]:
# 保存用ファイル名
PICKLE_FILE = 'summary_results.pkl'

if os.path.exists(PICKLE_FILE):
    # --- ファイルが存在する場合は復元 ---
    print(f'Loading existing summary results from {PICKLE_FILE}...')
    with open(PICKLE_FILE, 'rb') as f:
        summary_results = pickle.load(f)
    print(f'Successfully loaded {len(summary_results)} items.')

else:
    # --- ファイルが存在しない場合は Gemini で生成 ---
    print(f'{PICKLE_FILE} not found. Starting summarization with Gemini 2.5 Flash...')
    
    summary_results = {}

    for title, filename in pdf_dict.items():
        gcs_uri = f'gs://{BACKET}/pdf/{filename}'
        
        prompt = f'''
あなたは自治体の案内係です。提供されたPDF文書の内容を、後で検索（RAG）に利用するためのサマリーとしてまとめてください。
        
* 文書タイトル: {title}
        
* 以下のガイドラインに従ってください：
  1. 文書の主要な目的と対象者を明確にする。
  2. 手続きが必要な場合は、その名称や場所、必要なものをキーワードとして含める。
  3. 住民がこの文書を探す際に使いそうな検索クエリ（キーワード）を抽出して出力する。
  4. 以下の形式で500文字程度で出力する。（「：」の直後は改行しない。3つの項目の間は空行を1行入れる。）
    文書タイトル：<文書タイトル>
    文書サマリー：<文書のサマリー>
    検索キーワード：<キーワードを「、」で区切って並べる> 
'''

        try:
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=[
                    types.Part.from_uri(file_uri=gcs_uri, mime_type='application/pdf'),
                    prompt
                ]
            )
            
            summary_results[title] = {
                'filename': filename,
                'summary': response.text
            }
            print(f'Succeeded: {title}')
            
        except Exception as e:
            print(f'Failed: {title} - Reason: {e}')

    # 生成した結果を保存
    with open(PICKLE_FILE, 'wb') as f:
        pickle.dump(summary_results, f)
    
    print(f'\nAll summarization processes are complete and saved to {PICKLE_FILE}.')

!ls -lh {PICKLE_FILE}

# 内容の確認（最初の3件を表示）
if summary_results:
    for title in list(summary_results.keys())[:3]:
        print('\n' + '='*50)
        print(f'Summary sample of {title}.')
        print('='*50 + '\n')
        print(summary_results[title]['summary'])

summary_results.pkl not found. Starting summarization with Gemini 2.5 Flash...
Succeeded: 区長の挨拶
Succeeded: 引っ越しするとき
Succeeded: もくじ
Succeeded: いざというときに備えて
Succeeded: エリアシティウォーク
Succeeded: 公的施設等一覧
Succeeded: 主な区立施設の設備一覧
Succeeded: 相談
Succeeded: 区役所と特別出張所
Succeeded: 緊急のとき／安心安全
Succeeded: 手続き／届出
Succeeded: 健康／衛生
Succeeded: 高齢者福祉
Succeeded: 介護保険
Succeeded: 障害者福祉
Succeeded: その他の福祉
Succeeded: 子ども／教育
Succeeded: 文化／スポーツ
Succeeded: ごみ／環境／リサイクル
Succeeded: 生活環境
Succeeded: 住宅／まちづくり
Succeeded: 仕事／経営
Succeeded: 協働／身近な行政
Succeeded: わたしの便利帳2024年版　さくいん（索引）
Succeeded: 医療機関リスト

All summarization processes are complete and saved to summary_results.pkl.
-rw-r--r-- 1 jupyter jupyter 42K Dec 25 04:00 summary_results.pkl

Summary sample of 区長の挨拶.

文書タイトル：区長の挨拶

文書サマリー：この文書は、新宿区長吉住健一氏による「くらしのガイド」の紹介と、区の基本的な情報を提供するものです。「くらしのガイド」は、区民の安全で快適な暮らしを支援するための情報誌であり、行政サービスや医療機関に関する情報が2年に一度更新されます。巻頭では、大雨・台風・地震などの災害に備えるための防災情報と、新宿区の魅力を紹介する「エリアシティウォーク」として12の散策コースが特集されています。防災情報では、日頃の備え、災害情報の入手方法、発災時の行動について解説し、エリアシティウォークでは、各コー

### サマリーテキストから埋め込みベクトルを生成

In [4]:
# 保存用ファイル名
EMB_PICKLE_FILE = 'embeddings_dict.pkl'

if os.path.exists(EMB_PICKLE_FILE):
    # --- ファイルが存在する場合は復元 ---
    print(f'Loading existing embeddings from {EMB_PICKLE_FILE}...')
    with open(EMB_PICKLE_FILE, 'rb') as f:
        embeddings_dict = pickle.load(f)
    print(f'Successfully loaded embeddings for {len(embeddings_dict)} documents.')

else:
    # --- ファイルが存在しない場合は生成 ---
    print('Generating embeddings using gemini-embedding-001...')
    embeddings_dict = {}

    for title, data in summary_results.items():
        summary_text = data['summary']
        
        try:
            # gemini-embedding-001 を使用してベクトル化
            # task_type='RETRIEVAL_DOCUMENT' を指定
            res = client.models.embed_content(
                model='gemini-embedding-001',
                contents=summary_text,
                config=types.EmbedContentConfig(task_type='RETRIEVAL_DOCUMENT')
            )
            
            # ベクトル（数値配列）を取り出す
            vector = res.embeddings[0].values

            embeddings_dict[title] = {
                'filename': data['filename'],
                'embedding': vector
            }
            print(f'Succeeded:  {title}')
            
        except Exception as e:
            print(f'Failed: {title} - Reason: {e}')

    # 生成した結果を保存
    with open(EMB_PICKLE_FILE, 'wb') as f:
        pickle.dump(embeddings_dict, f)

    print(f'\nAll embedding processes are complete and saved to {EMB_PICKLE_FILE}.')

!ls -lh {EMB_PICKLE_FILE}

# データの詳細を確認（最初の3件）
if embeddings_dict:
    print('\n' + '='*50)
    print('Embedding Verification (First 10 elements)')
    print('='*50)

    # 最初の3つのキー（タイトル）を取得
    for title in list(embeddings_dict.keys())[:3]:
        sample_vector = embeddings_dict[title]['embedding']
        
        print(f'Document Title: {title}')
        print(f'Vector Dimension: {len(sample_vector)}')
        print(f'Vector (first 10): {sample_vector[:10]}')
        print('-' * 30)

Generating embeddings using gemini-embedding-001...
Succeeded:  区長の挨拶
Succeeded:  引っ越しするとき
Succeeded:  もくじ
Succeeded:  いざというときに備えて
Succeeded:  エリアシティウォーク
Succeeded:  公的施設等一覧
Succeeded:  主な区立施設の設備一覧
Succeeded:  相談
Succeeded:  区役所と特別出張所
Succeeded:  緊急のとき／安心安全
Succeeded:  手続き／届出
Succeeded:  健康／衛生
Succeeded:  高齢者福祉
Succeeded:  介護保険
Succeeded:  障害者福祉
Succeeded:  その他の福祉
Succeeded:  子ども／教育
Succeeded:  文化／スポーツ
Succeeded:  ごみ／環境／リサイクル
Succeeded:  生活環境
Succeeded:  住宅／まちづくり
Succeeded:  仕事／経営
Succeeded:  協働／身近な行政
Succeeded:  わたしの便利帳2024年版　さくいん（索引）
Succeeded:  医療機関リスト

All embedding processes are complete and saved to embeddings_dict.pkl.
-rw-r--r-- 1 jupyter jupyter 677K Dec 25 04:00 embeddings_dict.pkl

Embedding Verification (First 10 elements)
Document Title: 区長の挨拶
Vector Dimension: 3072
Vector (first 10): [-0.01997458003461361, -0.014920834451913834, 0.015888767316937447, -0.03366269916296005, -0.007585332728922367, 0.0005019793170504272, -0.003779264632612467, 0.0056771510280668736, 0.0036213

## コンシェルジュ AI を実装

### 質問文から関連する PDF を検索する関数

In [5]:
def search_relevant_documents(user_query, top_k=3):
        # 1. Search query を生成 (HyDEアプローチ)
    hyde_prompt = f'''
    ユーザーの質問に対して、自治体のガイドブックからどのような情報を探すべきか、
    回答に含まれるべき具体的なキーワードや内容を300文字程度で書き出してください。
    これは検索用のクエリとして使用します。
    
    ユーザーの質問: {user_query}
    '''
    
    hyde_response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=hyde_prompt
    )
    search_query = hyde_response.text

    # 2. search_query をベクトル化 (RETRIEVAL_QUERY)
    res = client.models.embed_content(
        model='gemini-embedding-001',
        contents=search_query,
        config=types.EmbedContentConfig(task_type='RETRIEVAL_QUERY')
    )
    query_vector = np.array(res.embeddings[0].values)

    # 3. コサイン類似度の計算
    similarities = []
    for title, data in embeddings_dict.items():
        doc_vector = np.array(data['embedding'])
        
        # コサイン類似度の数式: (A・B) / (|A|*|B|)
        score = np.dot(query_vector, doc_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(doc_vector))
        similarities.append((title, score))

    # 4. スコアの高い順にソート
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    return search_query, sorted_similarities[:top_k]

In [6]:
# --- テスト実行 ---
question = '最近引っ越してきたのですが、古い電子レンジを捨てるにはどうすればいいですか？'
print(f'User input: {question}')
search_query, results = search_relevant_documents(question)

print('\n--- Search query ---')
print(search_query)
print('\n--- Search result (top 3) ---')
for i, (title, score) in enumerate(results):
    filename = embeddings_dict[title]['filename']
    print(f'{i+1}. {title} (Score: {score:.4f}) - ファイル: {filename}')

User input: 最近引っ越してきたのですが、古い電子レンジを捨てるにはどうすればいいですか？

--- Search query ---
自治体のガイドブックから以下のキーワードで情報を探してください。

「**電子レンジ 処分方法**」「**粗大ごみ 出し方**」「**小型家電リサイクル**」「**不用品回収**」

特に以下の点について具体的な内容を確認します。

1.  **区分と分類**: 電子レンジが「粗大ごみ」に分類されるか、「小型家電リサイクル」の対象品目か、または「資源ごみ」など他の区分で回収されるか。
2.  **申し込み手続き**: 「戸別収集」の「予約方法」、必要な「手数料」や「費用」、支払い方法（「リサイクル券」の購入など）。
3.  **持ち込み**: 自治体の「清掃センター」や「指定引取場所」への「持ち込み」が可能か、その場合の「場所」「受付時間」「料金」。
4.  **その他**: 家電量販店や専門業者による「引取りサービス」の案内、具体的な「問い合わせ先」。

これらの情報から、ご自身の状況に合った最適な処分方法を見つけ出してください。

--- Search result (top 3) ---
1. ごみ／環境／リサイクル (Score: 0.7813) - ファイル: 000391199.pdf
2. もくじ (Score: 0.7208) - ファイル: 000391174.pdf
3. 相談 (Score: 0.7191) - ファイル: 000391180.pdf


### 質問文と検索された PDF から回答を生成する関数

In [7]:
def generate_final_answer(user_query, search_results):
    # 1. コンテキストとして渡すPDFのPartリストを作成
    # 検索結果（上位3件）からファイル名を取り出し、GCS URIを作成
    content_parts = []
    found_titles = []
    
    for title, score in search_results:
        filename = embeddings_dict[title]['filename']
        gcs_uri = f'gs://{BACKET}/pdf/{filename}'
        
        # PDFをPartとして追加
        content_parts.append(
            types.Part.from_uri(file_uri=gcs_uri, mime_type='application/pdf')
        )
        found_titles.append(title)

    # 2. 最終回答用のプロンプトを作成
    system_instruction = f'''
    あなたは自治体の親切な案内係です。提供された{len(found_titles)}つのPDF資料（{", ".join(found_titles)}）の内容に基づいて、
    ユーザーの質問に正確に、かつ分かりやすく答えてください。
    
    制約事項：
    - 資料に記載されている内容のみに基づいて回答してください。
    - 資料に情報がない場合は「提供された資料の中には、その質問に関する情報が見当たりませんでした」と正直に答えてください。
    - どの資料を参考にしたかも最後に付け加えてください。
    '''
    
    # 質問文を最後に追加
    content_parts.append(f'\n質問：{user_query}')

    # 3. Gemini 2.5 Flash による最終回答の生成
    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=content_parts,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction,
                temperature=0.2 # 安定した回答のため低めに設定
            )
        )
        return response.text
    except Exception as e:
        return f'回答生成中にエラーが発生しました: {e}'

In [8]:
# --- 全行程を統合した実行テスト ---

# 質問例
#question = '最近引っ越してきたのですが、粗大ゴミの出し方について教えてください。'
question = '最近引っ越してきたのですが、古い電子レンジを捨てるにはどうすればいいですか？'
print(f'User input: {question}')

# ステップ1: 関連ドキュメントの検索（上位3件）
search_query, search_results = search_relevant_documents(question, top_k=3)

# ステップ2: 最終回答の生成
final_answer = generate_final_answer(question, search_results)

print('\n--- Search query ---')
print(search_query)
print('\n--- Search result ---')
for i, (title, score) in enumerate(search_results):
    filename = embeddings_dict[title]['filename']
    print(f'{i+1}. {title} (Score: {score:.4f}) - ファイル: {filename}')

# --- 整形して表示 ---
print('\n--- Final answer ---')
display(Markdown(final_answer))

User input: 最近引っ越してきたのですが、古い電子レンジを捨てるにはどうすればいいですか？

--- Search query ---
自治体のガイドブックで探すべき情報とキーワード：

*   **「電子レンジ」の品目分類:**
    *   「粗大ごみ」「不燃ごみ」「小型家電リサイクル」のいずれに該当するかを確認します。
    *   キーワード：「電子レンジ」「粗大ごみ」「小型家電リサイクル」「ごみ分別」「品目別出し方」

*   **具体的な処分方法と手順:**
    *   「出し方」「収集方法」「持ち込み」の可否、およびその手続き。
    *   キーワード：「処分方法」「収集日」「持ち込み場所」「申し込み」

*   **費用と手数料:**
    *   処分にかかる料金や手数料の有無、支払い方法。
    *   キーワード：「手数料」「料金」「費用」

*   **注意事項:**
    *   取り外し方、指定場所、出す時間帯など。
    *   キーワード：「リサイクル」「家電」

これらのキーワードでガイドブックの索引や目次、PDF検索を行うと、適切な情報にたどり着きやすいでしょう。

--- Search result ---
1. ごみ／環境／リサイクル (Score: 0.7571) - ファイル: 000391199.pdf
2. もくじ (Score: 0.7292) - ファイル: 000391174.pdf
3. わたしの便利帳2024年版　さくいん（索引） (Score: 0.7240) - ファイル: 000391177.pdf

--- Final answer ---


新宿区へのお引越し、お疲れ様でした。古い電子レンジの捨て方についてご案内します。

電子レンジの捨て方は、その大きさによって異なります。

1.  **一辺の長さがおおむね30cmを超える場合**
    電子レンジが一辺の長さがおおむね30cmを超える場合は、「粗大ごみ（有料）」として出す必要があります。
    粗大ごみは事前申込制ですので、以下のいずれかの方法でお申し込みください。
    *   **インターネット：** 粗大ごみ受付センターのウェブサイトから（24時間受付）
    *   **電話：** 粗大ごみ受付センター 5304-8080（月～土、午前8時～午後7時）

2.  **一辺の長さがおおむね30cm以内の場合**
    電子レンジが一辺の長さがおおむね30cm以内の小型家電製品であれば、「金属・陶器・ガラスごみ」として出すことができます。
    この場合、ふたつきの容器か中身の見えるポリ袋に入れて、月2回の収集日に出してください。

どちらに該当するか、電子レンジのサイズをご確認ください。

---
参照資料：
*   ごみ／環境／リサイクル (P146)

## パーソナライゼーションの実装

In [9]:
def generate_personalized_answer(user_query, search_results, user_profile, past_history):
    # 1. コンテキストの構築（PDF、プロファイル、履歴をまとめる）
    content_parts = []

    # ユーザーのプロファイルと履歴をテキストパートとして追加
    user_context_text = f'''
    # ユーザープロファイル:
    {user_profile}

    # 過去の質問履歴:
    {past_history}
    '''
    content_parts.append(user_context_text)

    # 検索でヒットしたPDFを追加
    found_titles = []
    for title, score in search_results:
        filename = embeddings_dict[title]['filename']
        gcs_uri = f'gs://{BACKET}/pdf/{filename}'
        content_parts.append(
            types.Part.from_uri(file_uri=gcs_uri, mime_type='application/pdf')
        )
        found_titles.append(title)

    # ユーザーの質問を追加
    content_parts.append(f'\n質問：{user_query}')

    # 2. システム指示の更新
    system_instruction = f'''
    あなたは自治体の親切な案内係です。以下の手順で回答を作成してください。

    1. 導入メッセージ（挨拶）:
       ユーザープロファイルと過去の質問履歴を確認し、冒頭でユーザーに合わせた親しみやすい挨拶を行ってください。
       （例：以前の相談内容に触れる、ユーザーの状況を気遣うなど）
       ただし、不自然にならないよう、かつ個人情報に配慮した丁寧なトーンを維持してください。

    2. 本題の回答:
       提供された{len(found_titles)}つのPDF資料（{", ".join(found_titles)}）に基づき、質問に正確に答えてください。
       資料にない内容は答えないでください。

    3. 出典の明記:
       最後に参照した資料名を記載してください。
    '''

    # 3. Gemini による生成
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=content_parts,
        config=types.GenerateContentConfig(
            system_instruction=system_instruction,
            temperature=0.3
        )
    )
    return response.text

In [11]:
# --- 実行イメージ ---

# サンプルデータ（実際はDBやファイルから読み込む想定）
user_profile = '''
東京在住50代男性
出身地：大阪
名前：中川太郎
職業：IT企業に勤めるエンジニア。
家族：妻と小学生の娘が一人いる。
'''
past_history = '''
- 2024-12-10: 区内のIT勉強会について質問
- 2025-01-15: 子供の教育手当について質問
'''

question = '夏休みに娘と出かける区内のスポットを教えて。'

print(f'User input: {question}')

# 検索の実行
search_query, search_results = search_relevant_documents(question, top_k=3)

# パーソナライズされた回答の生成
final_answer = generate_personalized_answer(question, search_results, user_profile, past_history)

# 表示
print('\n--- Search query ---')
print(search_query)
print('\n--- Search result ---')
for i, (title, score) in enumerate(search_results):
    filename = embeddings_dict[title]['filename']
    print(f'{i+1}. {title} (Score: {score:.4f}) - ファイル: {filename}')
print('\n--- Final answer ---')
display(Markdown(final_answer))

User input: 夏休みに娘と出かける区内のスポットを教えて。

--- Search query ---
「夏休み」「子供向け」「親子」「家族向け」「キッズ」を主要キーワードとし、以下の情報を自治体のガイドブックから探してください。

*   **時期・目的別特集**: 「夏休み特集」「イベントカレンダー（夏季）」の項目。
*   **屋外施設**: 「公園（水遊びができる公園、大型遊具、自然体験）」「プール」「動物園」「植物園」「夏祭り」「花火大会」
*   **屋内施設**: 「博物館（子供向け展示、体験型プログラム）」「科学館」「図書館（読み聞かせ、イベント）」「文化施設」「ショッピングモール（キッズスペース、イベント）」
*   **体験・学習**: 「ワークショップ」「ものづくり体験」「食育イベント」
*   **その他**: 「雨の日でも楽しめるスポット」「涼しい場所」「無料スポット」「熱中症対策」

これらのキーワードで目次や索引、本文を検索し、具体的なスポット名、開催期間、対象年齢、料金、アクセス方法、事前予約の有無に関する記述を探します。

--- Search result ---
1. わたしの便利帳2024年版　さくいん（索引） (Score: 0.7378) - ファイル: 000391177.pdf
2. 文化／スポーツ (Score: 0.7373) - ファイル: 000391197.pdf
3. もくじ (Score: 0.7313) - ファイル: 000391174.pdf

--- Final answer ---


中川様、こんにちは！以前はIT勉強会や、お子様の教育手当についてお問い合わせいただき、ありがとうございました。娘さんも小学生になられて、夏休みのお出かけ先をお探しとのこと、お役に立てれば幸いです。

夏休みに娘さんと一緒に楽しめる区内のスポットについて、いくつかご紹介させていただきますね。

1.  **新宿コズミックセンター プラネタリウム**
    季節ごとに様々なプログラムを公開しており、お子様も楽しめる内容が期待できます。公開日時やプログラムの詳細は、「広報新宿」や財団のホームページでご確認いただけます。
    *   参照ページ：文化／スポーツ p.132

2.  **博物館・記念館**
    区内には歴史や文化に触れられる施設がいくつかございます。小中学生の入館料が無料になる日もありますので、ぜひご検討ください。
    *   **新宿歴史博物館**：区の歴史を学べます。小中学生は土・日曜日、祝日、都民の日が無料です。
        *   参照ページ：文化／スポーツ p.136
    *   **漱石山房記念館**：夏目漱石の旧居跡に建てられた記念館で、文学の世界に触れることができます。小中学生は通常展100円です。
        *   参照ページ：文化／スポーツ p.136
    *   **林芙美子記念館**：作家・林芙美子が住んでいた家を保存・公開しています。小中学生は土・日曜日、祝日、都民の日が無料です。
        *   参照ページ：文化／スポーツ p.136
    *   **佐伯祐三アトリエ記念館**、**中村彝アトリエ記念館**：洋画家のアトリエを公開しており、入館は無料です。
        *   参照ページ：文化／スポーツ p.137
    *   **新宿ミニ博物館**：区内の寺社文化財や伝統工芸などを公開しています。
        *   参照ページ：文化／スポーツ p.137

3.  **神田川ふれあいコーナー／神田川親水テラス**
    神田川の歴史や生き物について学習できる展示コーナーがあります。毎年夏の時期にはテラスが開放され、直接神田川に入ることができるようです。詳細については「広報新宿」でご確認ください。
    *   参照ページ：文化／スポーツ p.132

4.  **スポーツ施設**
    体を動かすのがお好きでしたら、区内のスポーツ施設もおすすめです。プールや体育室など、様々な施設で個人利用や教室が開催されています。
    *   **新宿コズミックスポーツセンター**
    *   **新宿スポーツセンター**
    *   **大久保スポーツプラザ**
    *   **四谷スポーツスクエア**
    *   **都新宿山吹高等学校プール/体育施設等開放**
    *   参照ページ：文化／スポーツ p.139

これらの施設は、娘さんの興味や活動内容に合わせてお選びいただけます。各施設の開館時間や休館日、料金、イベント情報などは、それぞれの施設にお問い合わせいただくか、ホームページ等で最新の情報をご確認ください。

楽しい夏休みになりますよう、心よりお祈り申し上げます。

---
参照資料：
*   わたしの便利帳2024年版 さくいん（索引）
*   文化／スポーツ
*   もくじ